In [2]:
import os,codecs
import numpy as np
import matplotlib.pyplot as plt

# source for extracting mnist data - 
# https://github.com/Ghosh4AI/Data-Processors/blob/master/MNIST/MNIST_Loader.ipynb

def get_int(b):   # CONVERTS 4 BYTES TO A INT
    return int(codecs.encode(b, 'hex'), 16)

def read_data():
    datapath = 'mnist/'
    files = os.listdir(datapath)
    data_dict = {}
    for file in files:
        if file.endswith('ubyte'):
            print('Reading ',file)
            with open (datapath+file,'rb') as f:
                data = f.read()
                type = get_int(data[:4])
                length = get_int(data[4:8])
                if (type == 2051):
                    category = 'images'
                    num_rows = get_int(data[8:12])
                    num_cols = get_int(data[12:16])
                    parsed = np.frombuffer(data,dtype = np.uint8, offset = 16) 
                    parsed = parsed.reshape(length,num_rows,num_cols)          
                elif(type == 2049):
                    category = 'labels'
                    parsed = np.frombuffer(data, dtype=np.uint8, offset=8)
                    parsed = parsed.reshape(length)                      
                if (length==10000):
                    set = 'test'
                elif (length==60000):
                    set = 'train'
                data_dict[set+'_'+category] = parsed
    return data_dict

data_dict = read_data()

X = data_dict["train_images"].reshape(60000,784)
Y = data_dict["train_labels"].reshape(60000,1)

# from tensorflow import keras
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from keras import initializers
from tensorflow.keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.layers import BatchNormalization

X = (X/255) - 0.5
Y = (Y/255) - 0.5

autoencoder = Sequential()
layer1 = Dense(512, input_dim = 784, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")
layer2 = Dense(128, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")(layer1)
layer3 = Dense(64, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")(layer2)
layer4 = Dense(128, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")(layer3)
layer5 = Dense(512, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")(layer4)
layer6 = Dense(784, trainable = True, activation = "relu", use_bias = True,
                kernel_initializer="random_normal", bias_initializer="random_normal")(layer5)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

autoencoder.add(layer1, layer2, layer3, layer4, layer5, layer6)

autoencoder.compile(
  optimizer=adam,
  loss='mse',
  metrics=['accuracy'],
)

autoencoder.summary()


Reading  t10k-images.idx3-ubyte
Reading  t10k-labels.idx1-ubyte
Reading  train-images.idx3-ubyte
Reading  train-labels.idx1-ubyte


ModuleNotFoundError: No module named 'tensorflow'